# Classification model Inference

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import fastai
from fastai.vision import *

import pandas as pd

import warnings
warnings.filterwarnings('ignore')
import numpy as np
from pathlib import Path

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score,precision_score,recall_score, accuracy_score
from sklearn.metrics import roc_curve,auc

In [2]:
import torch
torch.cuda.set_device(2)
torch.cuda.current_device()

2

# Results function 

In [3]:
def result_infe(pred_id,trues_id,total):  
#     from sklearn.metrics import accuracy_score, f1_score
    #accuracy
    acc = accuracy_score(pred_id, trues_id)
    print('Test set: accuracy is {0}.' .format(acc))

    #F1 score
    F1_validation=f1_score(pred_id,trues_id, average='weighted')
    print('Test set: F1_score is {0}.' .format(F1_validation))
    

    precision_validation=precision_score(pred_id, trues_id,average=None)
    recall_validation=recall_score(pred_id, trues_id,average=None)
    print('Test set: precision_score = {0},recall_score = {1} - HGG'.format(precision_validation[0],recall_validation[0])) 
    print('Test set: precision_score = {0},recall_score = {1} - LGG'.format(precision_validation[1],recall_validation[1]))
    
    total['acc'].append(acc)
    total['F1'].append(F1_validation)
    
    for i in range(2):
        total['pre'][i].append(precision_validation[i])
        total['rec'][i].append(recall_validation[i])
        
        
    # Majority of Votes:
    MV_labels = [];                                          # majority of votes decision list
    MV_predicted_labels = []
    for i in range(0, len(pred_id),5):               
        curr_patient = pred_id[i:i+5]                        # running on each patient
        occurances = Counter(curr_patient)                   # finding how many times each prediction was appeared
        max_key = max(occurances, key = occurances.get)      # extracting the most common appearance
        MV_labels.append(max_key) 
    #     print("for patient "+str(i)+' the *predicted* diagnosos is '+str(max_key))
        MV_predicted_labels.append(max_key)

    MV_True_labels = [];                                          # majority of votes decision list
    for i in range(0, len(trues_id),5):               
        curr_patient = trues_id[i:i+5]                        # running on each patient
        occurances = Counter(curr_patient)                   # finding how many times each prediction was appeared
        max_key = max(occurances, key = occurances.get)      # extracting the most common appearance
        MV_True_labels.append(max_key) 
    #     print("for patient "+str(i)+' the *True* diagnosos is '+str(max_key))

    lst=[]
    for i in range(len(MV_True_labels)):    
        if MV_True_labels[i] == MV_labels[i]:  
    #         print("patient" +str(i+1)+" TRUE prediction")
            lst.append("TRUE")
        else:  
    #         print("patient" +str(i+1)+" FALSE prediction")
            lst.append("FALSE")

    # Accuracy after majority of votes:
    acc_MV = (lst.count("TRUE")/len(MV_True_labels))
#     acc_MV2= accuracy_score(MV_True_labels, MV_predicted_labels)
#     print(acc_MV==acc_MV2)
    print('Test set: accuracy after majority of votes is {0}.' .format(acc_MV))

    # F1 score
    from sklearn.metrics import f1_score
    F1_validation=f1_score(MV_predicted_labels,MV_True_labels, average='weighted')
    print('Test set: F1-score after majority of votes is {0}.' .format(F1_validation))
    
    total['MV_acc'].append(acc_MV)
    total['MV_F1'].append(F1_validation)
    

# Load model

In [6]:
class FocalLoss(nn.Module):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets, **kwargs):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = self.alpha * ((1-pt)**self.gamma) * CE_loss
        return F_loss.mean()

In [7]:
def new_model(*args, **kwargs):
      
    model = models.resnet152(*args, **kwargs)
    model.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3, bias=False)
    return model

In [8]:
total_result={'acc':[],'F1':[],'pre':[[],[]],'rec':[[],[]],'MV_acc':[],'MV_F1':[]}

path= #BLINDED

test_images_path=Path(path+"/TEST_images")

df_test = pd.read_csv(path+"/sample_submission.csv")


for i_fold in range(1,6):
    print(f'fold {i_fold}//////////////////')
    
    bs=32
    imsize = 96
    
    learn = vision.load_learner(path=path+'./TRAIN/models', 
                                file=f'Classification_model{i_fold}.pkl')
    
    CLASS = 'label'
    ID = 'Image_id'

    test_data = (ImageList.from_df(df_test, test_images_path, suffix='.TIFF',convert_mode='RGBA')
            .split_none()
           .label_from_df(cols=CLASS)
           .databunch(bs=bs))
    
    #results 
    predictions = []
    predictionsC = []
    trues = df_test[CLASS].tolist()
    trues_check=[]
    for true, img in zip(trues, df_test[ID].tolist()):
        prediction = learn.predict(open_image(test_images_path+img+'.TIFF',convert_mode='RGBA'))
        predictionsC.append(prediction)
    
    
    pred_id = np.argmax(np.array([torch.nn.functional.softmax(a[2]).cpu().numpy() for a in predictionsC]), axis=1)
    class_id = test_data.classes
    cladd_id_dict = dict([(i,a) for (a,i) in enumerate(class_id)])
    trues_id = np.array([cladd_id_dict[a] for a in trues])
     
    result_infe(pred_id,trues_id,total_result)


fold 1//////////////////
You can deactivate this warning by passing `no_check=True`.
Test set: accuracy is 0.8363636363636363.


UnboundLocalError: local variable 'f1_score' referenced before assignment

# Results

In [40]:
print(u'Test set: accuracy is {0:.2f}\u00B1{1:.2f}.' .format(np.mean(total_result['acc']),np.std(total_result['acc'])))
print(u'Test set: F1-score is {0:.2f}\u00B1{1:.2f}.' .format(np.mean(total_result['F1']),np.std(total_result['F1'])))
print('\nFor HGG:')
print(u'Test set: precision_score = {0:.2f}\u00B1{1:.2f}'.format(np.mean(total_result['pre'][0]),np.std(total_result['pre'][0])))
print(u'Test set: recall_score = {0:.2f}\u00B1{1:.2f}'.format(np.mean(total_result['rec'][0]),np.std(total_result['rec'][0])))
print('\nFor LGG:')
print(u'Test set: precision_score = {0:.2f}\u00B1{1:.2f}'.format(np.mean(total_result['pre'][1]),np.std(total_result['pre'][1])))
print(u'Test set: recall_score = {0:.2f}\u00B1{1:.2f}'.format(np.mean(total_result['rec'][1]),np.std(total_result['rec'][1])))
print('\nAfter Majoroty of Votes:')
print(u'Test set: accuracy after majority of votes is {0:.2f}\u00B1{1:.2f}.' .format(np.mean(total_result['MV_acc']),np.std(total_result['MV_acc'])))
print(u'Test set: F1-score after majority of votes is {0:.2f}\u00B1{1:.2f}.' .format(np.mean(total_result['MV_F1']),np.std(total_result['MV_F1'])))

Test set: accuracy is 0.81±0.10.
Test set: F1-score is 0.79±0.12.

For HGG:
Test set: precision_score = 0.85±0.10
Test set: recall_score = 0.91±0.03

For LGG:
Test set: precision_score = 0.59±0.11
Test set: recall_score = 0.47±0.15

After Majoroty of Votes:
Test set: accuracy after majority of votes is 0.82±0.11.
Test set: F1-score after majority of votes is 0.81±0.13.
